In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import cairo

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten
from keras import backend as K
from keras.metrics import categorical_accuracy

from utility import generate_images_with_bboxes, display_image_with_bboxes

In [9]:
SEED = 100

num_imgs = 50000
img_size = 32
min_object_size = 8
max_object_size = 16
num_objects = 2

num_epochs = 50

In [15]:
def iou_calc(y_true, y_pred):
    y_true_f = K.flatten(y_true[0:4, 10:14])
    y_pred_f = K.flatten(y_pred[0:4, 10:14])
    intersection = K.sum(y_true_f * y_pred_f)
    return 2*(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))


def shape_acc(y_true, y_pred):
    y_true_shape = K.flatten(y_true[4:7, 15:18])
    y_pred_shape = K.flatten(y_pred[4:7, 15:18])
    return categorical_accuracy(y_true_shape, y_pred_shape)

def color_acc(y_true, y_pred):
    y_true_color = K.flatten(y_true[7:10, 18:])
    y_pred_color = K.flatten(y_pred[7:10, 18:])
    return categorical_accuracy(y_true_color, y_pred_color)

def iou_calc_loss(y_true, y_pred):
    return -iou_calc(y_true, y_pred)

def shape_loss(y_true, y_pred):
    y_true_shape = K.flatten(y_true[4:7, 15:18])
    y_pred_shape = K.flatten(y_pred[4:7, 15:18])
    return categorical_crossentropy(y_true_shape, y_pred_shape)

def color_loss():
    y_true_color = K.flatten(y_true[7:10, 18:])
    y_pred_color = K.flatten(y_pred[7:10, 18:])
    return categorical_crossentropy(y_true_color, y_pred_color)

In [16]:
def basic_model(input_shape=[32, 32, 3], output_shape=20):
    filter_size = 3
    pool_size = 2
    model = Sequential([
        Conv2D(32, (6, 6), input_shape=input_shape, activation='relu'),
        Conv2D(64, (filter_size, filter_size), activation='relu'),
        Conv2D(128, (filter_size, filter_size), activation='relu'), 
        Conv2D(128, (filter_size, filter_size), activation='relu'), 
        Flatten(),
        Dropout(0.4),
        Dense(256, activation='relu'), 
        Dropout(0.4), 
        Dense(output_shape)
    ])

    model.compile(optimizer='adadelta', loss=[iou_calc_loss, shape_loss, color_loss], metrics=[iou_calc, shape_acc, color_acc])
    return model

In [17]:
imgs, bboxes, shapes, colors = generate_images_with_bboxes(
    num_imgs,
    img_size,
    min_object_size,
    max_object_size,
    num_objects
)

In [18]:
# Standardize images
x = (imgs - 128.) / 255.

# Get one-hot arrays
colors_one_hot = to_categorical(colors)
shapes_one_hot = to_categorical(shapes)

y = np.concatenate([bboxes / img_size, shapes_one_hot, colors_one_hot], axis=-1).reshape(num_imgs, -1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=SEED)

In [19]:
y_train[0]
print(y_train.shape)

(40000, 20)


In [20]:
model = basic_model([32, 32, 3], 20)
model.fit(x_train, y_train, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)    

ValueError: When passing a list as loss, it should have one entry per model outputs. The model has 1 outputs, but you passed loss=[<function iou_calc_loss at 0x7fd29425a840>, <function shape_loss at 0x7fd29425aa60>, <function color_loss at 0x7fd29425ae18>]

In [ ]:
y_test = model.predict(x_test)

In [ ]:
print(y_test[0])

In [ ]:
i = 10

# "De-standardize"
image = x_test[i] * 255 + 128

bboxes = []
bboxes.append(y_test[i][0:4] * img_size)
bboxes.append(y_test[i][10:14] * img_size)

shapes = []
shapes.append(np.argmax(y_test[i][4:7]))
shapes.append(np.argmax(y_test[i][15:18]))

colors = []
colors.append(np.argmax(y_test[i][7:10]))
colors.append(np.argmax(y_test[i][18:]))

In [ ]:
display_image_with_bboxes(image, bboxes, shapes, colors)